# Rapport sur TP - Artus et Guilhem

## Partie 1 : copie du rapport intégré au code 

- Data train = taille images * nb images = 28 * 28 * nb images
- Label train = taille finale du vecteur * nb images  = 10 * nb images
- Logique similaire pour données de test
- Taille entrée = taille de w = data_train nb de colonnes * label_train*nombre de colonnes = 784 * 10
- Taille sortie = taille de b = 1 * label_train nb de colonnes = 1 * 10
- Taille de x = taille images * 5
- Taille de y = taille de 5 résultats = 5 * 10
- Taille de t = comme y
- Taille of grad = 5 * 10. Intuitivement celui-ci je ne l'ai pas, commment ce nb va-t-il augmenter quand nous allons appliquer la chain rule ?
- 

### Ce que donne le code : 
Taille entrée = 784, taille sortie = 10
 size of data_train : torch.Size([63000, 784])
 size of label_train : torch.Size([63000, 10])
 size of data_test : torch.Size([7000, 784])
 size of label_test : torch.Size([7000, 10])
 size of w : torch.Size([784, 10])
 size of b : torch.Size([1, 10])
 size of x : torch.Size([5, 784])
 size of y : torch.Size([5, 10])
 size of grad : torch.Size([5, 10])

## Partie 2 : Shallow Network - Grid Search

Modèle. Linear(784,k) → ReLU → Linear(k,10) ; sortie linéaire (logits), perte de classification (CrossEntropyLoss).

### Méthodologie.

Split 80/10/10 (train/val/test).

GridSearchCV (cv=2) sur 
lr ∈ {10e−2,10e−4}, 
nb_neuronnes_couche_caché ∈ {20,1000}
batch ∈ {16,128}
→ meilleur ~95.3% val

Affinage manuel (budget calcul limité) autour des meilleurs candidats, avec suivi val et early-stopping implicite par observation de la courbe.

### Hyperparamètres retenus.

η=0.005
nb_neuronnes_couche_caché = 500, batch = 64, 30 époques.
Scores obtenus : test ≈ 98.13% (meilleur epoch), validation = 97.70%.

### Influence observée.

η
η trop grand → oscillations; trop petit → convergence lente. 
η=0.005
η=0.005 donne le meilleur compromis.

nb_neuronnes_couche_caché : 20 sous-ajuste, 1000 commence à sur-ajuster; 500 maximise val.

Batch 64 stabilise l’optimisation (16 plus bruité, 128 un peu moins performant).

###  Remarques de bonne pratique.

Utiliser CrossEntropyLoss avec labels entiers (plus adapté que MSE sur one-hot).

Conserver un seed et répéter 2–3 runs si possible pour lisser l’aléa d’initialisation.

## Partie 3 : Deep Learning

Nous avons implémenté un réseau profond (au moins deux couches cachées) en utilisant PyTorch. L’architecture comporte trois couches cachées avec des tailles variables 
(a,b,c)
(a,b,c) et la fonction d’activation ReLU. La sortie est normalisée avec un Softmax.

Hyperparamètres fixés :

Taux d’apprentissage 
η=0.001
η=0.001

Batch size = 64

30 époques maximum avec arrêt précoce (si la précision test diminue)

Optimiseur : SGD, fonction de perte MSE

### Tests réalisés

Nous avons testé plusieurs architectures : (500,250,50), (50,250,500) et (250,250,250).

### Résultats :

Meilleure précision obtenue ≈ 97.4 % sur le test, atteinte en moins de 10 époques.

Les trois architectures donnent des résultats proches (entre 96.5 % et 97.4 %).

L’arrêt précoce évite l’overfitting après ~8–10 époques.

### Influence des hyperparamètres

Profondeur / largeur des couches : plus de couches permet de dépasser rapidement 96 %, mais élargir les couches n’apporte qu’un faible gain.

Taux d’apprentissage : fixé à 0.001, il assure une convergence rapide et stable.

Batch size : 64 est un compromis raisonnable entre rapidité et stabilité.

### Conclusion

Sur MNIST, un MLP profond atteint environ 97 % de précision en peu d’époques. La taille des couches a peu d’influence, alors que le choix des hyperparamètres d’entraînement (learning rate, batch size, arrêt précoce) joue un rôle plus important.

## Partie 4 : CNN 

### Choix du modèle
Conformément à l'ennoncé du TP, nous avons choisi d'implémenter un CNN en appliquant l'architecture LeNet. Notre travail a consisté à l'adpater à la taille de nos images, en prenant en compte les bonnes dimension. Pas besoin de padding dans notre cas car déjà inclu dans les données de base apparemment.

#### Choix des couches et dimensions associées : 
- Convolution 1 (+relu) : entrée en dimension 1*28*28 (car niveaux de gris uniquement donc pas *3 comme en couleur), sortie en 24*24*6 après convolution (kernel_size = 5, stride =1)
- Pooling 1 (+relu) : prends les données de Conv1 en entrée, sortie en 12*12*6 (kernel_size=2, stride=2)
- Convolution 2 (+ relu) : entrée de taille précédente, sortie en 8*8*16 (kernel_size=5, stride=1)
- Pooling 2 (+ relu) : entrée de taille précédente, sortie en 4*4*16 (kernel_size=2, stride=2)
- Fully connected 1 (Linear layer) : input: 16 * 4 * 4, output: 120 
- Fully connected 2 (Linear layer) : input: 120, output: 84 
- Output layer (Fully connected) ; input: 84, output: 10

Nous avons choisi une structure qui passe progressivement de 120 à 10 dimensions, car après test c'est ce qui semblait le mieux fonctionner.

## Hyperparamètres

Basé sur un grid-search, nous avons choisi : 
- Taille de batch : 64 (pour gagner en efficacité, batch size plus réduit n'apportait que peu à la qualité du modèle).
- Nombre d'époques : 30 (même si stabilisation à partir de 15 epochs, même si solution optimale trouvée autour de 25 on pourrait justifier qu'au delà de 15 notre modèle commence à over-fiter)
- Taux d’apprentissage : 0.005

## Résultats d'entraînement

L’évaluation de la précision sur l’ensemble test montre :  
- Une progression rapide durant les premières epochs, avec une précision passant de ~36% à plus de 90% en 4 epochs.  
- À partir de la 10e époque, la précision dépasse 95%.  
- Après environ 15 epochs, la précision atteint un premier plateau aux alentours de 97-98%, première convergence du modèle.  
- Au delà, les performances oscillent autour de la convergence avec semble-t-il une légère tendance à l'amélioration. Et ce sans sur-apprentissage marqué. Plus de puissance de calcul nous permettrait de vérifier si la performance continue vraiment à s'améliorer ou stagne au delà, mais pas besoin d'aller au delà de 15 epochs pour notre modèle.

| Epoch | Précision test |
|-------|----------------|
| 1     | 36%            |
| 4     | 91%            |
| 10    | 95%            |
| 15    | 97.6%          |
| 25    | 98.2%          |

## Conclusion

Le CNN utilisé permet d’obtenir une précision satisfaisante (>97%) sur MNIST, qui est certes un jeu de données réputé simple. Donc une architecture simple, sans trop besoin d'optimisation d'hyper paramètres suffit amplement.

L’entraînement est considéré comme terminé vers 15 epochs, où la précision se stabilise, ce qui permettrait d’économiser du temps de calcul sans perte significative de performance.  

A remarquer cependant que notre CNN n'offre pas d'augmentation significative de performances, par rapport aux modèles explorés plus tôt. Il faudrait donc sur ces données privilégier des méthodes plus simples et moins coûteuses en en énergie. Ce n'est pas si surprenant car les données MNIST sont réputées comme "simples", utiliser un CNN complèxe semble donc overkill sur un tel dataset.